# 01: Data Loading and Cleaning

## Objectives
- Load Excel data
- Basic quality checks
- Define Lead Quality primary metrics
- Derive core fields

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 2.1 Load Excel Data

In [ ]:
file_path = 'Analyst_case_study_dataset_1_(1) (1).xls'
df = pd.read_excel(file_path)

print(f"Data shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nFirst 5 rows:")
df.head()

## 2.2 Basic Quality Checks

In [ ]:
print(f"Total rows: {len(df)}")
print(f"Expected: ~3000")
print(f"Difference: {len(df) - 3000}")

vendor_id_col = df.columns[0]
print(f"\nVendorLeadID column: {vendor_id_col}")
print(f"Unique values: {df[vendor_id_col].nunique()}")
print(f"Total rows: {len(df)}")

if df[vendor_id_col].nunique() < len(df):
    duplicates = df[df[vendor_id_col].duplicated(keep=False)]
    print(f"\nFound duplicates: {len(duplicates)} rows")
    print(duplicates.head())
    df = df.drop_duplicates(subset=[vendor_id_col], keep='first')
    print(f"\nRows after deduplication: {len(df)}")

In [ ]:
call_status_col = df.columns[4] if len(df.columns) > 4 else df.columns[-1]
print(f"CallStatus column: {call_status_col}")

print(f"\nCallStatus unique values:")
print(df[call_status_col].value_counts())
print(f"\nNull count: {df[call_status_col].isna().sum()}")
print(f"\nAll unique values:")
print(df[call_status_col].unique())

date_col = df.columns[0]
print(f"\nDate column: {date_col}")

df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
print(f"\nNulls after datetime conversion: {df[date_col].isna().sum()}")
print(f"Date range: {df[date_col].min()} to {df[date_col].max()}")

In [ ]:
score_cols = [col for col in df.columns if 'score' in col.lower() or 'Score' in col]
print(f"Found Score columns: {score_cols}")

for col in score_cols:
    missing_pct = df[col].isna().sum() / len(df) * 100
    print(f"\n{col}:")
    print(f"  Missing count: {df[col].isna().sum()}")
    print(f"  Missing percentage: {missing_pct:.2f}%")
    if df[col].notna().sum() > 0:
        print(f"  Non-missing value distribution:")
        print(df[col].value_counts().sort_index())

## 3.1 Map CallStatus to 4 Groups + Binary Labels

In [ ]:
def map_call_status(status):
    if pd.isna(status):
        return 'unknown'
    
    status_str = str(status).strip().lower()
    
    if 'closed' in status_str:
        return 'closed'
    
    if any(x in status_str for x in ['ep sent', 'ep received', 'ep confirmed']):
        return 'good'
    
    if any(x in status_str for x in ['unable to contact', 'invalid profile', "doesn't qualify", "doesnt qualify"]):
        return 'bad'
    
    return 'unknown'

df['status_group'] = df[call_status_col].apply(map_call_status)

df['is_good'] = df['status_group'].isin(['closed', 'good']).astype(int)
df['is_closed'] = (df['status_group'] == 'closed').astype(int)
df['is_bad'] = (df['status_group'] == 'bad').astype(int)

print("Status Group distribution:")
print(df['status_group'].value_counts())
print("\nBinary label statistics:")
print(f"is_good=1: {df['is_good'].sum()} ({df['is_good'].mean()*100:.2f}%)")
print(f"is_closed=1: {df['is_closed'].sum()} ({df['is_closed'].mean()*100:.2f}%)")
print(f"is_bad=1: {df['is_bad'].sum()} ({df['is_bad'].mean()*100:.2f}%)")

## 0.1 Define Lead Quality Primary Metrics

In [ ]:
all_leads = len(df)
good_quality_count = df['is_good'].sum()
closed_count = df['is_closed'].sum()
bad_count = df['is_bad'].sum()

GoodQualityRate = good_quality_count / all_leads
CloseRate = closed_count / all_leads
BadRate = bad_count / all_leads

print("=" * 60)
print("LEAD QUALITY PRIMARY METRICS DEFINITION")
print("=" * 60)
print(f"\nTotal Leads: {all_leads}")
print(f"\n1. GoodQualityRate (Primary): {GoodQualityRate:.4f} ({GoodQualityRate*100:.2f}%)")
print(f"   Definition: (Closed + EP Sent + EP Received + EP Confirmed) / All")
print(f"   Numerator: {good_quality_count}")
print(f"\n2. CloseRate: {CloseRate:.4f} ({CloseRate*100:.2f}%)")
print(f"   Definition: Closed / All")
print(f"   Numerator: {closed_count}")
print(f"\n3. BadRate: {BadRate:.4f} ({BadRate*100:.2f}%)")
print(f"   Definition: (Unable to Contact + Invalid Profile + Doesn't Qualify) / All")
print(f"   Numerator: {bad_count}")
print("=" * 60)

## 3.2 Derive Time Fields

In [ ]:
df['date'] = df[date_col].dt.date
df['week'] = df[date_col].dt.to_period('W')
df['dow'] = df[date_col].dt.day_name()
df['day_index'] = (df[date_col] - df[date_col].min()).dt.days

print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Number of weeks: {df['week'].nunique()}")
print(f"\nWeekly statistics:")
print(df.groupby('week').size().head(10))

## Save Cleaned Data

In [ ]:
df.to_pickle('df_cleaned.pkl')
print("Data saved to df_cleaned.pkl")
print(f"\nFinal data shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())

## 4. Feature Engineering

### 4.1 Parse WidgetName

In [ ]:
widget_col = None
for col in df.columns:
    if 'widget' in col.lower() or 'WidgetName' in col:
        widget_col = col
        break

if widget_col:
    print(f"Found WidgetName column: {widget_col}")
    print(f"\nWidgetName sample values:")
    print(df[widget_col].head(10).tolist())
    
    def parse_widget_name(name):
        if pd.isna(name):
            return {'ad_size': None, 'dc_pages': None, 'design': None, 'bg_color': None}
        
        name_str = str(name).strip()
        parts = name_str.split('_')
        
        result = {
            'ad_size': None,
            'dc_pages': None,
            'design': None,
            'bg_color': None
        }
        
        if len(parts) >= 1:
            size = parts[0]
            if size in ['300250', '302252']:
                result['ad_size'] = '302252'
            else:
                result['ad_size'] = size
        
        if len(parts) >= 2:
            dc = parts[1].upper()
            if '1DC' in dc or '1' in dc:
                result['dc_pages'] = '1DC'
            elif '2DC' in dc or '2' in dc:
                result['dc_pages'] = '2DC'
        
        if len(parts) >= 3:
            result['design'] = parts[2]
        
        if len(parts) >= 4:
            result['bg_color'] = parts[3]
        
        return result
    
    widget_parsed = df[widget_col].apply(parse_widget_name)
    df['ad_size'] = widget_parsed.apply(lambda x: x['ad_size'])
    df['dc_pages'] = widget_parsed.apply(lambda x: x['dc_pages'])
    df['design'] = widget_parsed.apply(lambda x: x['design'])
    df['bg_color'] = widget_parsed.apply(lambda x: x['bg_color'])
    
    print("\nParsing results:")
    print(f"ad_size distribution:")
    print(df['ad_size'].value_counts())
    print(f"\ndc_pages distribution:")
    print(df['dc_pages'].value_counts())
else:
    print("WidgetName column not found")

### 4.2 Call Center vs Online Flag

In [ ]:
publisher_campaign_col = df.columns[7] if len(df.columns) > 7 else None
if publisher_campaign_col:
    print(f"PublisherCampaignName column: {publisher_campaign_col}")
    df['is_call_center'] = df[publisher_campaign_col].astype(str).str.contains('Call Center', case=False, na=False)
    print(f"\nCall Center distribution:")
    print(df['is_call_center'].value_counts())
    
    publisher_zone_col = df.columns[6] if len(df.columns) > 6 else None
    if publisher_zone_col:
        df['publisher_zone'] = df[publisher_zone_col]
        print(f"\nPublisherZoneName distribution:")
        print(df['publisher_zone'].value_counts().head(10))

### 4.3 Bin AddressScore and PhoneScore

In [ ]:
def bin_score(score_series, name):
    result = score_series.copy().astype(str)
    result[score_series.isna()] = 'missing'
    result[(score_series >= 1) & (score_series <= 2)] = '1-2'
    result[(score_series >= 3) & (score_series <= 4)] = '3-4'
    result[score_series == 5] = '5'
    return result

address_score_col = None
phone_score_col = None

for col in df.columns:
    if 'address' in col.lower() and 'score' in col.lower():
        address_score_col = col
    if 'phone' in col.lower() and 'score' in col.lower():
        phone_score_col = col

if address_score_col:
    df['address_score_bin'] = bin_score(df[address_score_col], 'AddressScore')
    print(f"AddressScore bin distribution:")
    print(df['address_score_bin'].value_counts())

if phone_score_col:
    df['phone_score_bin'] = bin_score(df[phone_score_col], 'PhoneScore')
    print(f"\nPhoneScore bin distribution:")
    print(df['phone_score_bin'].value_counts())

### 4.4 Branded vs Generic Flag

In [ ]:
advertiser_campaign_col = None
for col in df.columns:
    if 'advertiser' in col.lower() and 'campaign' in col.lower():
        advertiser_campaign_col = col
        break

if advertiser_campaign_col:
    print(f"AdvertiserCampaignName column: {advertiser_campaign_col}")
    df['is_branded'] = df[advertiser_campaign_col].astype(str).str.contains(
        'branded|creditsolutions', case=False, na=False
    )
    print(f"\nBranded distribution:")
    print(df['is_branded'].value_counts())

### 4.5 Other Features (Debt, State, Traffic Type)

In [ ]:
debt_col = None
for col in df.columns:
    if 'debt' in col.lower():
        debt_col = col
        break

if debt_col:
    print(f"Debt column: {debt_col}")
    df['debt_bin'] = pd.qcut(df[debt_col], q=3, labels=['Low', 'Medium', 'High'], duplicates='drop')
    print(f"\nDebt bin distribution:")
    print(df['debt_bin'].value_counts())

state_col = None
for col in df.columns:
    if 'state' in col.lower() and col.lower() != 'address':
        state_col = col
        break

if state_col:
    df['state'] = df[state_col]
    print(f"\nState distribution (Top 10):")
    print(df['state'].value_counts().head(10))

campaign_col = None
for col in df.columns:
    if 'campaign' in col.lower() and 'publisher' not in col.lower() and 'advertiser' not in col.lower():
        campaign_col = col
        break

if campaign_col:
    df['traffic_type'] = df[campaign_col].astype(str).str.contains('content', case=False, na=False)
    df['traffic_type'] = df['traffic_type'].map({True: 'content', False: 'search'})
    print(f"\nTraffic Type distribution:")
    print(df['traffic_type'].value_counts())